In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf

from sklearn.feature_selection import VarianceThreshold

Using TensorFlow backend.


In [3]:
data_p = pd.read_csv("drive/My Drive/INTERN/notebooks/SOCIS/sprint2/points.csv",dtype=object,error_bad_lines=False) 
data_p.head()
data_p["id"] = data_p["id"].map(str) +"_"+ data_p["dir"]
data_p.head()
# data_p.dtypes

,s.no,id,dir,path,date,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,x6,y6,x7,y7,x8,y8,x9,y9,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14,x15,y15,x16,y16,x17,y17,x18,...,x9981,y9981,x9982,y9982,x9983,y9983,x9984,y9984,x9985,y9985,x9986,y9986,x9987,y9987,x9988,y9988,x9989,y9989,x9990,y9990,x9991,y9991,x9992,y9992,x9993,y9993,x9994,y9994,x9995,y9995,x9996,y9996,x9997,y9997,x9998,y9998,x9999,y9999,x10000,y10000
0,1,ON_1_20181031_173504_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.03,0.2,0.03,0.2,0.03,0.2,0.03,0.2,0.03,0.0,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.0,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.2,0.03,0.2,0.03,0.0,0.03,0.2,...,0.0,0.15,0.4,0.15,0.2,0.14,0.2,0.15,0.2,0.14,0.0,0.14,0.2,0.15,0.2,0.15,0.2,0.14,0.2,0.14,0.4,0.15,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.14
1,2,OFF_4_20181031_173921_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.4,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.0,0.12,0.4,0.12,0.2,0.12,0.4,0.12,0.2,0.12,0.2,0.12,0.0,0.12,0.2,0.11,0.2,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.2,...,25.0,0.03,25.0,0.03,25.4,0.03,25.0,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.2,0.03,25.2,0.03,25.0,0.03,25.2,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03
2,3,OFF_2_20181031_173628_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.13,0.0,0.12,0.0,0.12,0.0,0.12,0.0,0.12,0.2,0.12,0.2,0.12,0.2,0.12,0.4,0.12,0.2,0.13,0.2,0.13,0.2,0.12,0.2,0.12,0.2,0.13,0.2,0.12,0.2,0.13,0.0,0.12,0.2,...,25.0,0.03,25.2,0.03,25.2,0.02,25.2,0.03,25.4,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.2,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.03,25.0,0.02,25.0,0.02,25.0,0.03
3,4,ON_3_20181031_173800_104,104,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,31/10/2018,0.2,0.03,0.4,0.02,0.4,0.04,0.2,0.03,0.4,0.03,0.2,0.03,0.2,0.04,0.0,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.4,0.03,0.2,0.03,0.2,0.03,0.4,0.03,0.0,0.03,0.2,0.03,0.2,...,0.2,0.14,0.2,0.14,0.0,0.14,0.0,0.14,0.2,0.14,0.2,0.14,0.4,0.14,0.4,0.14,0.2,0.14,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.2,0.14,0.2,0.15,0.2,0.14,0.0,0.14,0.2,0.14,0.2,0.14,0.2,0.14
4,5,OFF_4_20181102_085018_103,103,C:\Hilink\BB4A\RESULT\BBM2\BBM2\NOMINAL\MANUAL...,02/11/2018,0.2,0.15,0.2,0.16,0.2,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.2,0.14,0.0,0.15,0.2,0.15,0.2,0.14,0.2,0.15,0.2,0.15,0.0,0.15,0.2,0.15,0.2,0.15,0.2,...,25.0,0.05,25.2,0.06,25.2,0.05,25.0,0.06,25.2,0.06,25.0,0.05,25.0,0.06,25.0,0.06,25.0,0.06,25.0,0.06,25.2,0.06,25.0,0.06,24.8,0.06,25.0,0.06,25.0,0.06,25.0,0.05,25.0,0.06,25.0,0.05,25.2,0.05,25.0,0.05


In [4]:

data_v = pd.read_csv("drive/My Drive/INTERN/notebooks/SOCIS/sprint2/values.csv",dtype=object,error_bad_lines=False )
le = preprocessing.LabelEncoder()

data_v['power_state_spec'] = le.fit_transform(data_v['power_state_spec'].astype('str'))

data_v['power_state_value'] = le.fit_transform(data_v['power_state_value'].astype('str'))
data_v["id"] = data_v["id"].map(str) +"_"+data_v["dir"]
data_v.head()

,s.no,id,dir,_file_,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,1,ON_1_20181031_173504_104,104,HEGSE_72.HTM,1,89.990000000000,150.000000000000,405.992200000000,1,60,510.204,800,C,NC,C,C
1,2,OFF_2_20181031_173628_104,104,HEGSE_72.HTM,0,7.992000000000,30.000000000000,-56.000000000000,0,10,510.204,-100,C,C,C,C
2,3,ON_3_20181031_173800_104,104,HEGSE_72.HTM,1,89.950000000000,140.000000000000,405.992200000000,1,60,495.049,800,C,NC,C,C
3,4,OFF_4_20181031_173921_104,104,HEGSE_72.HTM,0,7.997000000000,30.000000000000,-58.000000000000,0,10,495.049,-100,C,C,C,C
4,5,ON_1_20181102_084600_103,103,HEGSE_72.HTM,1,56.650000000000,169.800000000000,405.992200000000,1,60,510.204,800,C,C,C,C


In [0]:
arr_v = data_v.to_numpy()
arr_p = data_p.to_numpy()

In [6]:
arr_v = arr_v[0:]
print(arr_v)
arr_p = arr_p[0:]
print(arr_p)

[['1' 'ON_1_20181031_173504_104' '104' ... 'NC' 'C' 'C']
 ['2' 'OFF_2_20181031_173628_104' '104' ... 'C' 'C' 'C']
 ['3' 'ON_3_20181031_173800_104' '104' ... 'NC' 'C' 'C']
 ...
 ['767' 'OFF_2_20181113_232240_113' '113' ... 'C' 'NC' 'C']
 ['768' 'ON_3_20181113_232417_113' '113' ... 'C' 'C' 'C']
 ['769' 'OFF_4_20181113_232541_113' '113' ... 'C' 'NC' 'C']]
[['1' 'ON_1_20181031_173504_104' '104' ... '0.14' '0.2' '0.14']
 ['2' 'OFF_4_20181031_173921_104' '104' ... '0.03' '25.0' '0.03']
 ['3' 'OFF_2_20181031_173628_104' '104' ... '0.02' '25.0' '0.03']
 ...
 ['822' 'OFF_2_20181113_232240_113' '113' ... '0.05' '24.9359375' '0.05']
 ['823' 'OFF_4_20181113_232541_113' '113' ... '0.05' '24.9046875' '0.05']
 ['824' 'ON_3_20181113_232417_113' '113' ... '0.16' '0.278125' '0.16']]


In [7]:
ON_list =[]
OFF_list = []
for i in range(len(arr_p)):
    s = arr_p[i][1]
    s = str(s)
    
#     print(type(st))
    if s.find("N") == -1:
        OFF_list.append(arr_p[i])
    
    else:
        ON_list.append(arr_p[i])
# calculating for ON
print(len(ON_list),"ON")
print(len(OFF_list),"OFF")
arr_on_p = np.array(ON_list)
print(arr_on_p)

426 ON
398 OFF
[['1' 'ON_1_20181031_173504_104' '104' ... '0.14' '0.2' '0.14']
 ['4' 'ON_3_20181031_173800_104' '104' ... '0.14' '0.2' '0.14']
 ['7' 'ON_3_20181102_084856_103' '103' ... '0.17' '0.2' '0.17']
 ...
 ['819' 'ON_3_20181117_171650_114' '114' ... '0.15' '0.32109375' '0.15']
 ['821' 'ON_1_20181113_232117_113' '113' ... '0.17' '0.284375' '0.17']
 ['824' 'ON_3_20181113_232417_113' '113' ... '0.16' '0.278125' '0.16']]


In [8]:
arr_on_p = np.delete(arr_on_p, 3,  axis=1)
arr_on_p_n = arr_on_p[:, 1::2]
arr_on_p_f = np.delete(arr_on_p_n, 1,  axis=1)
print(len(arr_on_p_f))
print(len(arr_on_p_f[0]))
print(arr_on_p_f[0])

426
10001
['ON_1_20181031_173504_104' '0.03' '0.03' ... '0.14' '0.14' '0.14']


In [0]:

arr_on_p_f[:,1:]= arr_on_p_f[:,1:].astype('float64')
# print((arr_on_p_f[413]))

In [10]:
data = arr_on_p_f

df=pd.DataFrame(data=data[0:,0:],index=[i for i in range(data.shape[0])],
                columns=['y'+str(i) for i in range(data.shape[1])])
df.head()
# df.dtypes


,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [11]:
print(df.shape)
# for j in range(10000):
#   var = "y"+str(j+1)
#   df[var].fillna(df[var].mean(), inplace=True)
df_no_miss = df.dropna()
print(df_no_miss.shape)
print(df.shape)
df_no_miss.head()

(426, 10001)
(423, 10001)
(426, 10001)


,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [12]:
arr_p_no = df_no_miss.to_numpy()
print(len(arr_p_no))

423


In [13]:
df1= df_no_miss.rename(index=str, columns={"y0": "id"})
df1.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9961,y9962,y9963,y9964,y9965,y9966,y9967,y9968,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.17,0.17,0.17,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17


In [14]:
print(df1.shape)
df2 = data_v
print(df2.shape)
# df2['revenue'] = df2.CET.map(df1.set_index('date')['revenue'])

# to find out number of times a id is repeating
# print(df1.id.value_counts())


# print(df1['id'].nunique())
# print(df2['id'].nunique())

# print(df1.duplicated(['id','y1','y7897']).sum())

# print(df2.duplicated(['id']).sum())
combine = (pd.merge(df1, df2, how='left', on='id'))
# print(df1.unique)
print(combine.shape)

(423, 10001)
(769, 16)
(423, 10016)


In [15]:
combine.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,s.no,dir,_file_,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1,104,HEGSE_72.HTM,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800,C,NC,C,C
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,3,104,HEGSE_72.HTM,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800,C,NC,C,C
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,7,103,HEGSE_72.HTM,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800,C,C,C,C
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,5,103,HEGSE_72.HTM,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800,C,C,C,C
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,9,102,HEGSE_72.HTM,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800,C,C,C,C


In [16]:
combine.iloc[:,0:10010].head()
k = combine.drop(['s.no','dir','_file_'], axis = 1) 

k.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,...,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA),power_state_N/NC,current_rise/fall_time_C/NC,current_stabilised_C/NC,current_max/min_C/NC
0,ON_1_20181031_173504_104,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800,C,NC,C,C
1,ON_3_20181031_173800_104,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800,C,NC,C,C
2,ON_3_20181102_084856_103,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,...,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800,C,C,C,C
3,ON_1_20181102_084600_103,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800,C,C,C,C
4,ON_1_20181102_085847_102,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800,C,C,C,C


In [17]:
input_1 = k.iloc[:,0:10009]
# filling the missing values

# print(input_1.iloc[:,10008])
miss = input_1.iloc[:,1:]
miss.head()

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,...,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA)
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.990000000000,150.000000000000,405.992200000000,1.0,60,510.204,800
1,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.950000000000,140.000000000000,405.992200000000,1.0,60,495.049,800
2,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,0.06,...,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.970000000000,166.300000000000,405.992200000000,1.0,60,495.049,800
3,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.650000000000,169.800000000000,405.992200000000,1.0,60,510.204,800
4,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.040000000000,170.000000000000,401.312500000000,1.0,60,510.204,800


In [18]:
input_1 = k.iloc[:,0:10009]
# filling the missing values

# print(input_1.iloc[:,10008])
miss = input_1.iloc[:,1:]
miss.head()


# miss.columns[miss.isna().any()].tolist()
# for j in range((miss.shape[0])):
#   if miss["current_rise/fall_time_value (mS)"].notnull() == True:
#     print(j)
  
for column in (miss.iloc[:,10000:]):
  su = 0
  div = 0
  for r in range(miss.shape[0]):
    if (pd.isna(miss[column][r]))== False:
      su = float(miss[column][r])+su

      div = div+1

  fin = float(su/div)

  miss[column].fillna(float(fin),inplace=True)
#
 ################TO CHECK IS THERE IS ANY NAN ############################## 
#   if miss[column].isnull().any():

    
#     print('{0} has {1} null values'.format('current_rise/fall_time_value (mS)', miss['current_rise/fall_time_value (mS)'].isnull().sum()))
#   else:
#     print("no")
    
    


# if(pd.isna(miss["current_rise/fall_time_value (mS)"][23]))== True:


# print(miss.dtypes)
#########converting ever
# thing into float
miss =miss.astype('float64')
# print(miss.dtypes)
miss.head()


,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,...,y9969,y9970,y9971,y9972,y9973,y9974,y9975,y9976,y9977,y9978,y9979,y9980,y9981,y9982,y9983,y9984,y9985,y9986,y9987,y9988,y9989,y9990,y9991,y9992,y9993,y9994,y9995,y9996,y9997,y9998,y9999,y10000,power_state_value,current_rise/fall_time_value (mS),current_stabilised_value (mA),current_max/min_value (mA),power_state_spec,current_rise/fall_time_spec (mS),current_stabilised_spec (mA),current_max/min_spec (mA)
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.15,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.15,0.14,0.15,0.14,0.14,0.15,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.99,150.0,405.9922,1.0,60.0,510.204,800.0
1,0.03,0.02,0.04,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14,0.15,0.14,0.14,0.14,0.14,0.14,1.0,89.95,140.0,405.9922,1.0,60.0,495.049,800.0
2,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.05,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.05,0.06,0.05,0.05,0.06,0.06,0.06,0.06,0.05,0.05,0.05,0.06,...,0.16,0.17,0.17,0.17,0.16,0.16,0.16,0.17,0.16,0.17,0.16,0.17,0.16,0.16,0.17,0.16,0.16,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,1.0,36.97,166.3,405.9922,1.0,60.0,495.049,800.0
3,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.17,0.17,0.16,0.16,0.17,0.17,0.17,0.17,0.17,0.16,0.17,1.0,56.65,169.8,405.9922,1.0,60.0,510.204,800.0
4,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,...,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,1.0,37.04,170.0,401.3125,1.0,60.0,510.204,800.0


In [0]:
import pandas as pd
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import robust_scale

scaler_min_x = MinMaxScaler()
scaler_min_y = MinMaxScaler()

scaler_norm_x = Normalizer()
scaler_norm_y = Normalizer()

scaler_stan_x = StandardScaler()
scaler_stan_y = StandardScaler()

scalar_qt_x =QuantileTransformer(output_distribution='uniform')
scalar_qt_y =QuantileTransformer(output_distribution='uniform')
       

In [20]:
rand_na = miss
# print(miss.shape)
input_1_arr = rand_na.to_numpy()
input_1_arr[:,:]= input_1_arr[:,:].astype('float64')

X = input_1_arr[:,0:10000]
Y = input_1_arr[:,10002:10004]
# print(X.shape)
# print(Y.shape)
# print(Y)
y=np.reshape(Y, (-1,1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

# ######minmax
scaler_min_x = MinMaxScaler().fit(X_train)
scaler_min_y = MinMaxScaler().fit(y_train)

X_minmax_train = scaler_min_x.transform(X_train)
Y_minmax_train = scaler_min_y.transform(y_train)


# print(X)
# print(Y)
#####standard

scaler_stan_x = StandardScaler().fit(X_train)
scaler_stan_y = StandardScaler().fit(y_train)


X_stan_train = scaler_stan_x.transform(X_train)
Y_stan_train = scaler_stan_y.transform(y_train)

#######normlised
scaler_norm_x = Normalizer().fit(X_train)
scaler_norm_y = Normalizer().fit(y_train)


X_norm_train = scaler_norm_x.transform(X_train)
Y_norm_train = scaler_norm_y.transform(y_train)


# ################qt

scaler_qt_x =  QuantileTransformer(output_distribution='normal').fit(X_train)
scaler_qt_y =  QuantileTransformer(output_distribution='normal').fit(y_train)


X_qt_train = scaler_qt_x.transform(X_train)
Y_qt_train = scaler_qt_y.transform(y_train)


##robust
print(X_train.shape)
print(y_train.shape)
X_train_t = X_train.transpose()
y_train_t = y_train.transpose()
print(X_train.shape,"after")
print(y_train.shape,"after")
scaler_rob_x = RobustScaler().fit(X_train_t)
scaler_rob_y = RobustScaler().fit(y_train_t)


X_rob_train = scaler_rob_x.transform(X_train_t)
Y_rob_train = scaler_rob_y.transform(y_train_t)

X_rob_train = X_rob_train.transpose()
Y_rob_train = Y_rob_train.transpose()

print(X_rob_train.shape)
print(Y_rob_train.shape)
# print(Y_rob_train)

##robustscale
# print(X_train.shape)
# scaler_robsc_x= robust_scale(X_train, axis=1, with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)
# scaler_robsc_y= robust_scale(y_train, axis=1, with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)



# X_robsc_train = scaler_robsc_x
# Y_robsc_train = scaler_robsc_y
# print(X_robsc_train.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


(338, 10000)
(338, 2)
(338, 10000) after
(338, 2) after
(338, 10000)
(338, 2)


In [0]:
# from sklearn.decomposition import PCA
# # # pca - keep 90% of variance


# # print(X_norm)

# # print(Y_norm)
# pca = PCA(n_components = 30)
# pca_fit =  pca.fit(X_rob_train)
# X_new = pca_fit.transform(X_rob_train)
# print(X_new.shape)
# # principal_df = pd.DataFrame(data = principal_components)
# # print(principal_df.head())

# var_exp = pca.explained_variance_ratio_
# print(var_exp.shape)

In [22]:

from sklearn.decomposition import FactorAnalysis

transformer = FactorAnalysis(n_components=30, random_state=0)
factor_fit = transformer.fit(X_rob_train)
X_new = factor_fit.transform(X_rob_train)
X_new.shape

(338, 30)

In [0]:
# from sklearn.decomposition import FastICA

# transformer = FastICA(n_components=30, random_state=0)
# ica_fit = transformer.fit(X_stan_train)
# X_new = ica_fit.transform(X_stan_train)
# X_new.shape


In [0]:

# from sklearn.manifold import Isomap

# transformer = Isomap(n_components=50)

# iso_fit = transformer.fit(X_stan_train)
# X_new = iso_fit.transform(X_stan_train)
# X_new.shape


In [0]:


def baseline_model_30(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(28, activation='relu', 
                    kernel_initializer = 'he_normal', 
                    input_shape=(30,)))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(30, activation='relu',
#                     kernel_initializer = 'he_normal'))
#       model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(12, activation='relu',
                    kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(9, activation='relu',
#                     kernel_initializer = 'he_normal'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='linear', 
                    kernel_initializer='he_normal'))
    model.compile(loss = 'mse', optimizer=optimizer, metrics=['mae'])
#     model.summary()
    return model





In [0]:

def train_data_nn(X_train, y_train):
    
    np.random.seed(42)
    # create model
    estimator = KerasRegressor(build_fn=baseline_model_30, epochs=200, batch_size=5, verbose=0)
    kfold = KFold(n_splits=5, random_state=None )
    results = cross_val_score(estimator, X_train, y_train, cv=kfold)  
    print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return estimator

In [25]:
model = baseline_model_30()

estimator = train_data_nn(X_new, Y_rob_train)

print(X_new.shape)
print(y_train.shape)
history = estimator.fit(X_new,  Y_rob_train, epochs=200, batch_size=5,  verbose=1, validation_split=0.0)

W0714 19:41:25.172850 139891445606272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 19:41:25.195217 139891445606272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 19:41:25.199691 139891445606272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0714 19:41:25.278434 139891445606272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0714 19:41

Standardized: -0.09 (0.03) MSE
(338, 30)
(338, 2)
Epoch 1/200
338/338 [==============================] - 1s 4ms/step - loss: 1.9136 - mean_absolute_error: 1.0966
Epoch 2/200
338/338 [==============================] - 0s 1ms/step - loss: 1.2228 - mean_absolute_error: 0.8993
Epoch 3/200
338/338 [==============================] - 0s 1ms/step - loss: 0.7930 - mean_absolute_error: 0.7127
Epoch 4/200
338/338 [==============================] - 0s 1ms/step - loss: 0.5509 - mean_absolute_error: 0.5833
Epoch 5/200
338/338 [==============================] - 0s 1ms/step - loss: 0.3903 - mean_absolute_error: 0.4824
Epoch 6/200
338/338 [==============================] - 0s 1ms/step - loss: 0.2953 - mean_absolute_error: 0.4200
Epoch 7/200
338/338 [==============================] - 0s 1ms/step - loss: 0.2762 - mean_absolute_error: 0.3916
Epoch 8/200
338/338 [==============================] - 0s 1ms/step - loss: 0.2175 - mean_absolute_error: 0.3382
Epoch 9/200
338/338 [==============================] -

In [0]:
def visualize_learning_curve(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('model mean_absolute_error')
    plt.ylabel('mean_absolute_error')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [27]:
#row wise'

print(X_test.shape)
X_test_t = X_test.transpose()
print(X_test_t.shape)
scaler_rob_x = RobustScaler().fit(X_test_t)
X_new_test_t = scaler_rob_x.transform(X_test_t)
X_new_test = X_new_test_t.transpose()
print(X_new_test.shape)

y_test_t = y_test.transpose()
scaler_rob_y = RobustScaler().fit(y_test_t)
Y_new_test_t = scaler_rob_y.transform(y_test_t)
Y_new_test = Y_new_test_t.transpose()

X_new_test = factor_fit.transform(X_new_test)
print(X_new_test.shape)





(85, 10000)
(10000, 85)
(85, 10000)
(85, 30)


In [0]:
# # print(X_test.shape)
# # X_test_t = X_test.transpose()
# # print(X_test_t.shape)
# # scaler_rob_x = RobustScaler().fit(X_test)
# X_new_test = scaler_stan_x.transform(X_test)
# # X_new_test = X_new_test_t.transpose()
# print(X_new_test.shape)

# # y_test_t = y_test.transpose()
# # scaler_rob_y = RobustScaler().fit(y_test_t)
# Y_new_test = scaler_stan_y.transform(y_test)
# # Y_new_test = Y_new_test_t.transpose()

# X_new_test = pca_fit.transform(X_new_test)
# print(X_new_test.shape)

# results = model.evaluate(X_new_test, Y_new_test)
# print('loss: ', results[0])
# print('mse: ', results[1])

# print('accuracy',results[2])

# visualize_learning_curve(history)

In [29]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math
from sklearn.metrics import max_error

pred = estimator.predict((X_new_test))
print(pred.shape)
# print(pred[:,1])
# print(Y_new_test[:,1])
mse = (mean_squared_error(Y_new_test,pred))
# print(Y_new_test[80])
# print("L")
# print(pred[80])
print(mse)
# Xnew = scaler_norm_x.inverse_transform(Y_new_test[:,3])
# print(rmse)
# ynew = scaler_stan_y.inverse_transform(pred) 

#normalisation
# mul = math.sqrt(np.sum(y_test[58]**2))
# print("L")
# print(y_test[58])
# print(Y_new_test[58]*mul)
# print(pred[58]*mul)


# print(y_test[5])
# print(X_test[5])
# X  = factor_fit.transform(scaler_stan_x.transform(X_test[5].reshape(1, -1)))

# print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
# print(scaler_stan_y.inverse_transform(Y_new_test[50]))
# print(scaler_stan_y.inverse_transform(estimator.predict(X)))



# Xnew = scaler_stan_x.inverse_transform(Xnew)

# from sklearn.metrics import explained_variance_score
# y_true = Y_new_test
# y_pred= pred
# v = explained_variance_score(y_true, y_pred, multioutput='uniform_average') 
# print(v)
# print(max_error(y_true, y_pred))

# test_df = pd.DataFrame({'y_pred': pred})    
# return test_df

(85, 2)
0.16602291274446807


In [30]:
#row wise


for i in range(len(y_test)):
  
  print(y_test[i],"ytest[i]")
  
  
#   print(X_test[i])
  X_test_t= X_test[i].transpose()
#   print(X_test_t.reshape(-1, 1).shape,"X_test fitted ")
#   print(X_test_t.shape)
  scaler_rob_x = RobustScaler().fit((X_test_t.reshape(-1, 1)))
#   print(X_test_t.reshape(-1, 1))                            
  X = (scaler_rob_x.transform(X_test_t.reshape(-1, 1)))
#   print(X.shape,"X shape")
  I = factor_fit.transform(X.transpose())
#   print(I.shape,"I")
  pred = estimator.predict(I)
#   print(pred.shape,"pred.shape")
  
  y_test_ti = y_test[i].transpose()
#   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
  scaler_rob_y = RobustScaler().fit(y_test_ti.reshape(-1, 1))
  final_t = scaler_rob_y.inverse_transform(pred.reshape(-1, 1))
                                          
#   print(final_t.shape)
  final = final_t.transpose()
                                          
  print(final[0])
#   Y_new_test_t = scaler_rob_y.transform(y_test_ti)
#   Y_new_test = Y_new_test_t.transpose()

#   X= X.tranpose()
#   # print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
#   # print(scaler_stan_y.inverse_transform(Y_new_test[50]))
#   pred = (estimator.predict(X))
# #   print(scaler_stan_y.inverse_transform(estimator.predict(X)))
#   mul = math.sqrt(np.sum(y_test[i]**2))
# #   print("L")
# #   print(y_test[58])
# #   print(Y_new_test[58]*mul)
# #   print(pred[58]*mul)
#   pred = pred.tranform()
#   l = scaler_rob_y.inverse_transform(pred)
#   l=l.transpose()
# #   l = (estimator.predict(X))*mul
#   print(l,"l")
  h = abs(final-y_test[i])
#   print(h,"h")
  o=np.divide(h,y_test[i])
  print(o*100,"percentage") 


[150.1    401.3125] ytest[i]
[144.86499 407.91086]
[[3.48768139 1.64419453]] percentage
[170.4    627.6563] ytest[i]
[169.12372 628.47406]
[[0.74899163 0.13028788]] percentage
[140.     589.5938] ytest[i]
[138.11617 591.1255 ]
[[1.34559631 0.25978704]] percentage
[-200.     -202.6406] ytest[i]
[-202.43459 -200.20479]
[[-1.21729279 -1.20203542]] percentage
[149.3    401.3125] ytest[i]
[137.40945 413.50766]
[[7.96419669 3.03881885]] percentage
[ 26.33768194 498.06189793] ytest[i]
[ 14.949973 510.48026 ]
[[43.23732385  2.49333612]] percentage
[100.     562.1094] ytest[i]
[ 95.72238 565.6359 ]
[[4.27761841 0.62737348]] percentage
[135.     566.4063] ytest[i]
[135.34239 566.7962 ]
[[0.25362368 0.06883815]] percentage
[110.     631.1719] ytest[i]
[124.67091 616.6594 ]
[[13.33719427  2.2992906 ]] percentage
[-200.     -202.6406] ytest[i]
[-202.43459 -200.20479]
[[-1.21729279 -1.20203542]] percentage
[140.    578.125] ytest[i]
[129.59764 587.2509 ]
[[7.43025643 1.57853674]] percentage
[140.   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide


[157.29585 405.19827]
[[1.7514974  0.96826605]] percentage
[120.     637.3125] ytest[i]
[121.73868 634.4037 ]
[[1.44889832 0.45641871]] percentage
[140.     405.9922] ytest[i]
[149.15811 397.2893 ]
[[6.54150827 2.14361098]] percentage
[170.     401.3125] ytest[i]
[165.20743 406.67572]
[[2.81916001 1.33641993]] percentage
[155.2    401.3125] ytest[i]
[170.34872 386.56665]
[[9.76077601 3.67440576]] percentage
[162.     401.3125] ytest[i]
[174.6646  389.09433]
[[7.8176522 3.0445526]] percentage
[-99.5    363.3594] ytest[i]
[-99.605774 361.12247 ]
[[-0.10630545  0.61562545]] percentage
[129.8    732.0782] ytest[i]
[131.00105 729.99414]
[[0.92531037 0.28467715]] percentage
[130.     627.7657] ytest[i]
[153.19331 607.8829 ]
[[17.84101046  3.16723683]] percentage
[ 63.3    608.1719] ytest[i]
[ 67.16475 606.0757 ]
[[6.10544889 0.34467499]] percentage
[-9.8   -3.875] ytest[i]
[-9.339683  -4.3325315]
[[ -4.69711654 -11.80726328]] percentage
[160.     401.3125] ytest[i]
[156.15894 405.8625 ]
[[2.

In [31]:
####creating new X,Y
X1 = input_1_arr[:,0:10000]
Y1 = input_1_arr[:,10000:10004]

X=X1
Y=Y1[:,2:4]
# print(Y)
Y_new = np.zeros((Y.shape[0],2))
for i in range(len(Y)):
  
  print(Y[i],"y")
#   print(Y.shape)
  
  

  X_t= X[i].transpose()
#   print(X_test_t.reshape(-1, 1).shape,"X_test fitted ")
#   print(X_test_t.shape)
  scaler_rob_x = RobustScaler().fit((X_t.reshape(-1, 1)))
#   print(X_test_t.reshape(-1, 1))                            
  Xi = (scaler_rob_x.transform(X_t.reshape(-1, 1)))
#   print(X.shape,"X shape")
  I = factor_fit.transform(Xi.transpose())
#   print(I.shape,"I")
  pred = estimator.predict(I)
#   print(pred.shape,"pred.shape")
  
  Y_ti =Y[i].transpose()
#   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
  scaler_rob_y = RobustScaler().fit(Y_ti.reshape(-1, 1))
  final_t = scaler_rob_y.inverse_transform(pred.reshape(-1, 1))
                                          
#   print(final_t.shape)
  final = final_t.transpose()
                                          
  print(final[0])

  h = abs(final-Y[i])
#   print(h,"h")
  o=np.divide(h,Y[i])
  print(o*100,"percentage") 
  
  Y_new[i]=final[0]
#   print(Y[i],"Y")
#  print(Y_new[i])

# print(Y_new[:,0])




[150.     405.9922] y
[164.4873  391.61328]
[[9.65820312 3.54167365]] percentage
[140.     405.9922] y
[152.70609 393.7551 ]
[[9.07577515 3.01412282]] percentage
[166.3    405.9922] y
[160.81773 411.0559 ]
[[3.29661229 1.24724273]] percentage
[169.8    405.9922] y
[159.78178 416.7042 ]
[[5.90000939 2.63847264]] percentage
[170.     401.3125] y
[182.45874 389.80972]
[[7.32867073 2.86628926]] percentage
[165.6    401.3125] y
[159.2078  406.53464]
[[3.86002766 1.30126459]] percentage
[0.14      0.4013125] y
[0.1305748  0.41097924]
[[6.73229098 2.40878147]] percentage
[0.14      0.4013125] y
[0.12838678 0.41325668]
[[8.2951567  2.97627784]] percentage
[150.     648.7813] y
[172.3583 629.5628]
[[14.90553792  2.96224549]] percentage
[145.     607.4063] y
[154.63588 599.66766]
[[6.64543415 1.27404612]] percentage
[150.     624.5469] y
[140.83345 635.9049 ]
[[6.11103312 1.81859957]] percentage
[155.     630.7813] y
[152.26611 634.44934]
[[1.76379788 0.58150754]] percentage
[145.     625.6719] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


[-11.296251 -10.109385]
[[-12.96251297 -11.37016792]] percentage
[-11.6    -10.4531] y
[-11.510824 -10.541749]
[[-0.76875687 -0.84806422]] percentage
[-10.     -10.9219] y
[-10.84998  -10.071504]
[[-8.49980354 -7.78615773]] percentage
[120.     654.4063] y
[122.527336 651.17487 ]
[[2.10611343 0.49379633]] percentage
[120.     641.3125] y
[118.086365 642.35596 ]
[[1.59469604 0.16270649]] percentage
[130.     662.0157] y
[126.135994 665.44653 ]
[[2.97231234 0.51824046]] percentage
[120.     642.4063] y
[133.27176 629.5139 ]
[[11.05979919  2.00688941]] percentage
[120.     642.8438] y
[115.54095 648.34686]
[[3.71587753 0.85604976]] percentage
[120.2    658.3282] y
[131.27103 648.97925]
[[9.21050467 1.42010504]] percentage
[190.     401.3125] y
[182.2506  409.55988]
[[4.07863416 2.05510057]] percentage
[186.1    401.3125] y
[187.13252 400.5744 ]
[[0.55482138 0.18392105]] percentage
[2.63376819e+01 5.29080000e-03] y
[ 6.4766912 19.92902  ]
[[7.54090308e+01 3.76573092e+05]] percentage
[ 26.3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


[ 63.141182 461.47882 ]
[[139.73705085   7.34508648]] percentage
[ 26.33768194 498.06189793] y
[ 58.972527 467.39908 ]
[[123.90932764   6.15642748]] percentage
[ 26.33768194 498.06189793] y
[ 61.04664 464.42822]
[[131.78440163   6.75291072]] percentage
[ 26.33768194 498.06189793] y
[ 15.377128 510.22867 ]
[[41.61548582  2.44282294]] percentage
[ 26.33768194 498.06189793] y
[  9.65528 515.6855 ]
[[63.34043317  3.53843327]] percentage
[ 26.33768194 498.06189793] y
[ 10.572059 514.8784  ]
[[59.8595704   3.37639159]] percentage
[ 26.33768194 498.06189793] y
[ 63.140785 461.47925 ]
[[139.73554453   7.3450007 ]] percentage
[ 26.33768194 498.06189793] y
[ 63.00106 448.0754 ]
[[139.20503189  10.03620016]] percentage
[ 26.33768194 498.06189793] y
[ 62.921547 461.72202 ]
[[138.90313156   7.2962583 ]] percentage
[ 26.33768194 498.06189793] y
[ 63.142418 461.47772 ]
[[139.7417436    7.34530706]] percentage
[ 26.33768194 498.06189793] y
[ 14.1147375 510.429    ]
[[46.4085809   2.48304231]] percenta

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


[123.68941 636.79626]
[[3.07450612 0.59523562]] percentage
[ 26.33768194 498.06189793] y
[ 63.141182 461.47882 ]
[[139.73705085   7.34508648]] percentage
[ 26.33768194 498.06189793] y
[ 63.14315 461.47693]
[[139.74452449   7.34546637]] percentage
[130.     662.9844] y
[125.8513  667.39014]
[[3.19130531 0.66453098]] percentage
[-9.1    -3.2656] y
[-8.648106  -3.7146957]
[[ -4.96587229 -13.75231786]] percentage
[-9.8   -3.875] y
[-9.339683  -4.3325315]
[[ -4.69711654 -11.80726328]] percentage
[145.     613.7657] y
[137.07654 621.15967]
[[5.46445649 1.20468902]] percentage
[145.     619.2969] y
[156.70573 607.7688 ]
[[8.07292017 1.86148214]] percentage
[145.     620.8594] y
[142.36249 623.8466 ]
[[1.81897394 0.48114253]] percentage
[140.     600.6094] y
[136.73616 604.7564 ]
[[2.33131409 0.69046683]] percentage
[140.     598.0625] y
[128.22382 610.35583]
[[8.41156006 2.0555268 ]] percentage
[140.     596.0469] y
[135.076 601.551]
[[3.51713998 0.92343831]] percentage
[180.     626.3907] y


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide


 y
[-69.70656 514.5522 ]
[[-16.17759705   2.29982979]] percentage
[-60.     489.0625] y
[-53.137695 489.45947 ]
[[-11.43717448   0.08117013]] percentage
[170.     612.2969] y
[166.3603 613.3653]
[[2.1409966  0.17448976]] percentage
[170.4  630.75] y
[169.3065 631.3033]
[[0.64172342 0.08771838]] percentage
[145.     615.8282] y
[128.97652 631.3963 ]
[[11.05067812  2.52799421]] percentage
[145.     620.8125] y
[151.94069 613.91327]
[[4.78668213 1.11132282]] percentage
[140.     595.5938] y
[139.50966 596.90576]
[[0.3502437  0.22027793]] percentage
[145.     618.3282] y
[154.949  608.6377]
[[6.86138285 1.56721053]] percentage
[140.     596.5782] y
[134.82103 601.1342 ]
[[3.69926453 0.76369138]] percentage
[140.     597.8907] y
[119.20902 619.4367 ]
[[14.8506982   3.60366979]] percentage
[-140.      318.9219] y
[-138.68205  320.34906]
[[-0.94139099  0.44749516]] percentage
[-180.      264.4531] y
[-101.84511  186.35757]
[[-43.41938443  29.53095484]] percentage
[-1.3    -2.0156] y
[-1.01752

In [0]:
X1 = X1
Y1 = Y1
# print(Y1)

# print(Y)

# print(Y_new[:,0])

Y1[:,2]= Y_new[:,0]
Y1[:,3]= Y_new[:,1]
# print(Y1)

In [33]:
X1_new = np.concatenate((X1,Y1[:,2:4]),axis=1)
print(X1_new.shape)
Y1_new = Y1[:,0:2]
# print(Y1_new)

(423, 10002)


In [0]:
#train and test

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X1_new, Y1_new, test_size=0.20)


In [35]:
#standardise


# ######minmax
scaler_min_x = MinMaxScaler().fit(X_train_c)
scaler_min_y = MinMaxScaler().fit(y_train_c)

X_minmax_train = scaler_min_x.transform(X_train_c)
Y_minmax_train = scaler_min_y.transform(y_train_c)


# print(X)
# print(Y)
#####standard

scaler_stan_x = StandardScaler().fit(X_train_c)
scaler_stan_y = StandardScaler().fit(y_train_c)


X_stan_train = scaler_stan_x.transform(X_train_c)
Y_stan_train = scaler_stan_y.transform(y_train_c)

#######normlised
scaler_norm_x = Normalizer().fit(X_train_c)
scaler_norm_y = Normalizer().fit(y_train_c)


X_norm_train = scaler_norm_x.transform(X_train_c)
Y_norm_train = scaler_norm_y.transform(y_train_c)


# ################qt

scaler_qt_x =  QuantileTransformer(output_distribution='normal').fit(X_train_c)
scaler_qt_y =  QuantileTransformer(output_distribution='normal').fit(y_train_c)


X_qt_train = scaler_qt_x.transform(X_train_c)
Y_qt_train = scaler_qt_y.transform(y_train_c)


##robust
# print(X_train.shape)
# print(y_train.shape)
# X_train_t = X_train.transpose()
# y_train_t = y_train.transpose()
# print(X_train.shape,"after")
# print(y_train.shape,"after")
scaler_rob_x = RobustScaler().fit(X_train_c)
scaler_rob_y = RobustScaler().fit(y_train_c)


X_rob_train = scaler_rob_x.transform(X_train_c)
Y_rob_train = scaler_rob_y.transform(y_train_c)

# X_rob_train = X_rob_train.transpose()
# Y_rob_train = Y_rob_train.transpose()

# print(X_rob_train.shape)
# print(Y_rob_train.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (338). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [36]:
#apply PCA on X1_new
transformer = FactorAnalysis(n_components=30, random_state=0)
factor_fit = transformer.fit(X_rob_train[:,0:10000])
X_new1 = factor_fit.transform(X_rob_train[:,0:10000])
print(X_new1.shape)
X_new1 = np.concatenate((X_new1,X_rob_train[:,10000:10002]),axis=1)
print(X_new1.shape)
# print((X_rob_train[:,0:10000].shape))

(338, 30)
(338, 32)


In [0]:


def baseline_model_31(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(28, activation='relu', 
                    kernel_initializer = 'he_normal', 
                    input_shape=(32,)))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
#     model.add(Dense(30, activation='relu',
#                     kernel_initializer = 'he_normal'))
#       model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(12, activation='relu',
                    kernel_initializer = 'he_normal'))
    model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(9, activation='relu',
                    kernel_initializer = 'he_normal'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='linear', 
                    kernel_initializer='he_normal'))
    model.compile(loss = 'mse', optimizer=optimizer, metrics=['mae'])
#     model.summary()
    return model



In [0]:
def train_data_nn_1(X_train, y_train):
    
    np.random.seed(42)
    # create model
    estimator = KerasRegressor(build_fn=baseline_model_31, epochs=200, batch_size=5, verbose=0)
    kfold = KFold(n_splits=5, random_state=None )
    results = cross_val_score(estimator, X_train, y_train, cv=kfold)  
    print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    return estimator

In [39]:
model1 = baseline_model_31()

estimator1 = train_data_nn_1(X_new1, Y_rob_train)

# print(X_new.shape)
# print(y_train.shape)
history = estimator1.fit(X_new1,  Y_rob_train, epochs=250, batch_size=5,  verbose=1, validation_split=0.0)

Standardized: -3.21 (1.19) MSE
Epoch 1/250
338/338 [==============================] - 2s 6ms/step - loss: 5.8099 - mean_absolute_error: 1.6532
Epoch 2/250
338/338 [==============================] - 0s 1ms/step - loss: 4.7595 - mean_absolute_error: 1.3690
Epoch 3/250
338/338 [==============================] - 0s 1ms/step - loss: 4.0338 - mean_absolute_error: 1.1964
Epoch 4/250
338/338 [==============================] - 0s 1ms/step - loss: 3.9596 - mean_absolute_error: 1.1224
Epoch 5/250
338/338 [==============================] - 0s 1ms/step - loss: 3.7444 - mean_absolute_error: 1.0518
Epoch 6/250
338/338 [==============================] - 0s 1ms/step - loss: 3.8168 - mean_absolute_error: 1.0525
Epoch 7/250
338/338 [==============================] - 0s 1ms/step - loss: 3.6892 - mean_absolute_error: 1.0012
Epoch 8/250
338/338 [==============================] - 0s 1ms/step - loss: 3.7606 - mean_absolute_error: 0.9860
Epoch 9/250
338/338 [==============================] - 0s 1ms/step - loss

In [40]:
#row wise'


print(X_test_c.shape)

X_rob_test_c = scaler_rob_x.transform(X_test_c)

print(X_rob_test_c.shape)

Y_rob_test_c = scaler_rob_y.transform(y_test_c)


X_new_test_c = factor_fit.transform(X_rob_test_c[:,0:10000])
print(X_new_test.shape)


X_new_test_c = np.concatenate((X_new_test_c,X_rob_test_c[:,10000:10002]),axis=1)
print(X_new_test_c.shape)



(85, 10002)
(85, 10002)
(85, 30)
(85, 32)


In [41]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
import math
from sklearn.metrics import max_error

pred_c = estimator1.predict((X_new_test_c))
print(pred_c.shape)

mse = (mean_squared_error(Y_rob_test_c,pred_c))

print(mse)

(85, 2)
1.9474177798022727


In [42]:
for i in range(len(y_test_c)):
  
  print(y_test_c[i],"ytest[i]")
#   print(X_test_c[i].shape)
  X_c = (scaler_rob_x.transform(X_test_c[i].reshape(1, -1)))

  I = factor_fit.transform(X_c[:,0:10000])
  I = np.concatenate((I,X_c[:,10000:10002]),axis=1)
#   print(I.shape,"I shape")

  pred_c = estimator1.predict(I)
#   print(pred_c.shape,"pred_c.shape")
  
 

  
  final = scaler_rob_y.inverse_transform(pred_c.reshape(1, -1))
                                          
  final[0][0]= np.round(final[0][0])
                                          
  print(final[0],"final")

  h = abs(final[0]-y_test_c[i])
#   print(h,"h")
  o=np.divide(h,y_test_c[i])
  print(o*100,"percentage") 


[ 1.   52.27] ytest[i]
[ 0.99999326 50.96929   ] final
[6.73532486e-04 2.48844139e+00] percentage
[ 1.   49.09] ytest[i]
[ 0.9999924 50.573784 ] final
[7.62939453e-04 3.02257868e+00] percentage
[ 1.   51.35] ytest[i]
[ 0.9999928 53.034157 ] final
[7.21216202e-04 3.27976008e+00] percentage
[ 1.   36.85] ytest[i]
[ 1.0000072 40.284027 ] final
[7.15255737e-04 9.31893378e+00] percentage
[ 1.  52.4] ytest[i]
[ 0.9999946 54.658707 ] final
[5.42402267e-04 4.31050890e+00] percentage
[ 1.   87.09] ytest[i]
[ 0.99999183 57.863113  ] final
[8.16583633e-04 3.35594059e+01] percentage
[ 1.   50.75] ytest[i]
[ 0.999989 51.696285] final
[1.10268593e-03 1.86460147e+00] percentage
[ 1.   51.05] ytest[i]
[ 0.9999893 50.61869  ] final
[0.00107288 0.84487661] percentage
[ 1.   44.44] ytest[i]
[ 0.999988 49.41216 ] final
[1.19805336e-03 1.11884765e+01] percentage
[ 1.   56.39] ytest[i]
[ 0.99999166 50.955402  ] final
[8.34465027e-04 9.63752017e+00] percentage
[ 1.         50.40596493] ytest[i]
[ 1.0000012 5

In [0]:
# for i in range(len(y_test)):
#   print(y_test[i],"ytest[i]")
#   X=X_test[i]
# #   print(X.reshape(1, -1).shape)
#   pred = estimator.predict(pca_fit.transform(X.reshape(1, -1)))
#   print(pred)
#   print("-------")

In [0]:
# from sklearn.metrics import r2_score
  
# r2_score(Y_new_test[:,:], pred[:,:], multioutput='variance_weighted') 

In [0]:


# for i in range(len(y_test)):
  
#   print(y_test[i],"ytest[i]")
  
  
# #   print(X_test[i])
# #   X_test[i]= X_test[i].transpose()
# #   print(X_test[i].reshape(-1, 1).shape,"X_test fitted ")
# #   scaler_rob_x = RobustScaler().fit((X_test[i].reshape(-1, 1)))
# #   print(X_test[i].reshape(-1, 1))                            
# #   X = (scaler_stan_x.transform(X_test[i].reshape(-1, 1)))
  
# #   pred = estimator.predict(X)
# #   print(pred.shape,"pred.shape")
#   print()
# #   y_test_ti = y_test[i].transpose()
# #   print(y_test_ti.reshape(-1, 1).shape,"ytest tranposed shape")
# #   scaler_rob_y = RobustScaler().fit(y_test_ti.reshape(-1, 1))
#   final = scaler_stan_y.inverse_transform(pred[i].reshape(-1, 1))
                                          
# #   print(final_t.shape)
# #   final = final_t.transpose()
                                          
#   print(final)
# #   Y_new_test_t = scaler_rob_y.transform(y_test_ti)
# #   Y_new_test = Y_new_test_t.transpose()

# #   X= X.tranpose()
# #   # print(scaler_stan_y.transform(y_test[50].reshape(1, -1)))
# #   # print(scaler_stan_y.inverse_transform(Y_new_test[50]))
# #   pred = (estimator.predict(X))
# # #   print(scaler_stan_y.inverse_transform(estimator.predict(X)))
# #   mul = math.sqrt(np.sum(y_test[i]**2))
# # #   print("L")
# # #   print(y_test[58])
# # #   print(Y_new_test[58]*mul)
# # #   print(pred[58]*mul)
# #   pred = pred.tranform()
# #   l = scaler_rob_y.inverse_transform(pred)
# #   l=l.transpose()
# # #   l = (estimator.predict(X))*mul
# #   print(l,"l")
# #   h = abs(l-y_test[i])
# # #   print(h,"h")
# #   o=np.divide(h,y_test[i])
# #   print(o*100,"percentage")

In [0]:
# print(history.history.keys())
# # "Loss"
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')

# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [0]:
# # "mean_squared_error"
# plt.plot(history.history['mean_squared_error'])
# plt.plot(history.history['val_mean_squared_error'])
# plt.title('model mean_squared_error')
# plt.ylabel('mean_squared_error')

# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()

In [0]:
#  # "acc"
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])

# plt.title('model acc')
# plt.ylabel('acc')

# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')

# plt.show()